In [ ]:
import sys
sys.path.append(".")
sys.path.append("..")
sys.path.append("tests")
from IPython.display import IFrame
import importlib
import numpy as np
import matplotlib.pyplot as plt
from pointparticles.single_particle_dynamics import PointParticle
%matplotlib notebook

# Part 2: Pendulum with an elastic string
This is the notebook for Part 2 of Project 1:
- The spring system will be implemented in `simple_pendulum.py` following the outline described in this notebook.
- This notebook will show you how you can test the implementation yourself.

## 1. An overview: Forces in the pendulum system
Like the 1-dimensional spring system, our model for the pendulum system contains 2 forces: gravity ( $\vec{G}$ ) and the tension force ( $\vec{T}$ ) from the string. Because we're allowing the string to be elastic (meaning we can change its length by either compressing or extending it), we'll model the string as a very stiff spring (see figure below).

#### Question: Is the spring constant ($k$) for a stiff spring high or low? (Hint: What does $k$ measure?)

Answer:

Unlike in the 1-dimensional spring system, the forces are now *vectors*, meaning they can have one component in the $x$-direction and one component in the $y$-direction. This isn't a problem, however, as Newton's laws of motion are just as valid in vector form:

$$\sum \vec{F}=m\vec{a}$$

**NOTE**: In the 1-dimensional system we used $x$ to represent the vertical dimension (*upwards*). Whether we call this dimension $x$, $y$ or $donkey$ doesn't matter as long as we're consistent. Now that we're working in 2 dimensions, we'll follow standard notation and define the $x$-direction to point horizontally, and define the $y$-direction to point vertically. See the figure for details.

<img src="tikzfigures/2dpendulum.png" alt="PendulumSystemDiagram" style="width: 800px;"/>

### Investigation: Expressing the Net Force
Our first job is to find a way to express the net force. Our vectors have 2 components, so we can write them like this:

$$\vec{r}=\left(\array{x\\y}\right),\quad \vec{F}=\left(\array{F_x\\F_y}\right)\quad\text{or}\quad\vec{r}=(x,y),\quad \vec{F}=(F_x,F_y)$$

Whether we choose to write the vectors as column vectors (left) or row vectors (right) doesn't matter *as long as we're consistent*.

#### Task: Find a way to write $\vec{G}$ as above.

Answer:

Before continuing, make sure that you can see the parallel between the 1-dimensional spring we studied in part 1 and the spring-like elastic string we're modelling here. To test your understanding, try to imagine the 2-dimensional pendulum system depicted in the figure above in a gravityless environment (i.e., $\vec{G}=0$). How would this pendulum behave?

#### Question: How would the gravityless system behave?

Answer:

Let's now tackle the tension force. Recall from part 1 that the spring force was given by $S=-k(x-x_{eq})$. We're now going to generalize this expression to account for the direction of $\vec{T}$ by swapping $x$ and $x_{eq}$ for vectors $\vec{r}$ and $\vec{r}_{eq}$:

$$\vec{T}=-k(\vec{r}-\vec{r}_{eq})$$

#### Task: Find a unit vector $\vec{u}$ that points from the attachment point of the pendulum to the mass (Hint: $\vec{r}$ and $\vec{r}_b$ look very suspicious.)
#### Note: a unit vector is a vector of length 1.

Answer:

#### Task: Find $\vec{r}_{eq}$ by combining $\vec{r}_b$, $\vec{u}$ and $L$.

#### (Hint: $\vec{r}_{eq}$ is the *natural* position of the pendulum when the string is pointing in the direction of $\vec{u}$. If you're having trouble finding $\vec{r}_{eq}$, try finding $x_{eq}$ from $x_b$ and $L$ in the 1-dimensional spring system.)

Answer:

We're now finally ready to combine everything into one neat equation:

$$\sum\vec{F}=\vec{G}+\vec{T}=\vec{G}-k(\vec{r}-\vec{r}_{eq})$$

## 2. NumPy Vectors
**OBS**: You're not expected to remember all this right here and now. Check it out now and just move on to the next section whenever you feel ready. You can always come back and get whatever you need later.

To write vectors in Python we *can* use the built-in `list` class. Let's look at an example:

In [ ]:
r1 = [1, 2]  # position vector with x=1 and y=2
r2 = [2, 3]  # position vector with x=2 and y=3

# prepare sum vector
r_sum = [0,0]

# fill with the sum of the x components and y components (separately)
r_sum[0] = r1[0] + r2[0]
r_sum[1] = r1[1] + r2[1]

# we can't do "r_sum = r1 + r2" as we would on paper
# if you don't see why, try printing r1+r2

print(r1, r2, r_sum)

Great! It works. But it's super tedious to type out `r_sum[0] = r1[0] + r2[0]` every time we want to sum the $x$-components of a vector. Luckily, Python's `NumPy` package comes to the rescue. Here's the same code but using `NumPy` (btw. `NumPy` is usually pronounced *numpee*):

In [ ]:
# setup arrays by converting the lists to arrays
r1 = np.array([1,2])
r2 = np.array([2,3])

r_sum = r1 + r2
print(r1, r2, r_sum)

Boom! Much better. We can now even write our vector equations like we would on paper (meaning we don't have to add each component separately like we do for lists). Better yet, operations with `NumPy` arrays are written in `C` so they're **much** faster than using lists. This isn't a problem for our simple example here, but if you're performing $>10^6$ operations the difference can be huge.

There are several ways to generate an array. These are the simplest (i.e., best):

In [ ]:
# generate an array from a list
l = [1,2,3]
x = np.array(l)   # or simply  np.array([1,2,3])

# generate an array of length 4 filled with zeros
y = np.zeros(4)

# generate an array with 5 linearly spaced elements from 0 to 1
z = np.linspace(0, 1, 5)

print(x,y,z)

When you have created a `NumPy` array you can access the elements like this:

In [ ]:
# prepare sample array
x = np.linspace(1,10,10)
print(x)

# get element at index
print(x[0])   # this is the 1st element in the list
print(x[3])   # this is the -> 4th <- (NOT 3rd) element in the array
print(x[-1])  # this is the last element in the array

# range of indices
print(x[0:3])   # get elements from index 0 to and NOT including index 3
print(x[0:-1])  # get every element except the element at the last index
print(x[:-1])   # get every element except the element at the last index
print(x[1:])    # get every element except the element at the first index

# number of elements in an array
print(len(x))
print(x.size)

By accessing the elements in an array we can also change them:

In [ ]:
# prepare arrays
x = np.linspace(1,10,10)
y = np.linspace(3,1,3)   # linspace can also go backwards!
print(x,y)

# edit x
x[3] = -1
x[4:7] = y

# edit the entirety of y
y = y**2

# print results
print(x,y)

When working with vector equations, `NumPy` arrays are delightful to work with. Here's a couple of **awesome** mathematical properties of `NumPy` arrays.

In [ ]:
# setup example arrays
x = np.array([1,1])
y = np.array([-1,2])
print(x,y)

# array arithmetic
print(x+y, x-y, x*y, x/y)

# arrays can also be raised to a power term by term
print(x**2, y**2)

# arrays also work with ordinary numbers
print(x+2, x-2, x*2, x/2)

# finding the length of vectors : |x| = np.sqrt( x[0]^2 + y[0]^2 )
print(np.linalg.norm(x), np.linalg.norm(y))

# mathematical functions of arrays (happens at each entry separately)
print(np.sin(x))   # sin(x)
print(np.cos(x))   # cos(x)
print(np.exp(x))   # e^x

# summation and products
print(np.sum(x))
print(np.prod(x))

`NumPy`comes prepared with *everything you will ever need* for mathematical programming. Mathematical functions, algorithms, linear algebra, differential equations, complex numbers, $n$-dimensional tensors, etc. etc. You need it, `NumPy`'s got it. In order to use `NumPy` arrays like I did here, you need the statement:
```
import numpy as np
```

## 3. Setting up the Pendulum System
You're now going to make a functor called `NetPendulumForce` in the same style as you did with `NetSpringForce` in Part 1. The requirements are the same as before, except the `xb` argument should now be replaced by `rb`, which is an array with two elements. The default value of `rb` should be `np.array([0,1])`.

Like before we need to check that `NetPendulumForce` actually works before we can start analyzing the results generated by the functor. How can we check that it works? Well, we know how normal pendulums work. So can we relate our elastic pendulum to a normal pendulum?

#### Task: What '*value*' of $k$ do we need to get a normal pendulum from the elastic pendulum? (Hint: What happens when you change $k$?)

Answer:

### Reproducing a normal pendulum
Use your functor with an *appropriate* value of $k$ (Pro tip: Trail and error) in order to simulate a normal pendulum using `PointParticle`. Use these parameters and initial conditions:

In [ ]:
# pendulum parameters
m = 1
k = # fill
L = 1
g = 9.81
rb = np.array([0,1])

# initial conditions for the motion
theta = -np.pi/3
u = np.array([np.cos(theta), np.sin(theta)])  # if you're curious what this does, Google polar coordinates
r0 = rb + L*u
v0 = np.zeros(2)

# integration parameters
dt = 1e-4  # btw. this means 1 * 10^(-4)
N = 1e5    # so this means   1 * 10^(5)

You may have to adjust the `integration parameters`. If you get really weird results, try *decreasing* `dt` and *increasing* `N`.

In [ ]:
# you need to run this to update NetPendulumForce if you change anything in simple_pendulum.py
import simple_pendulum
from simple_pendulum import NetPendulumForce
simple_pendulum = importlib.reload(simple_pendulum)

# implement the pendulum here

# plot results with PointParticle's plot_2D_path() method here

You have hopefully gotten something that looks like the path of a pendulum. If you're having trouble, double check your `__call__` method to see if you have implemented $\sum\vec{F}$ correctly. If you're sure its correct, go back and double check your formulas from section 1.

### Testing the normal pendulum
What's great about testing your elastic pendulum code using the normal pendulum is that we know exactly how normal pendulums work. Specifically, we know that if the amplitude of the pendulum is small, the period is approximately given by:

$$T=2\pi\sqrt{\frac{L}{g}}$$

#### Task
Plot the $x$ and $y$ components of the normal pendulum motion and use the plots to judge whether our simulated pendulum obeys the period equation above (if it does so only approximately that's fine). You may have to find a suitable combination of `dt` and `N` so that it's easy to read off the graph.

In [ ]:
# implement the motion here

# .plot(0) <- this plots the x component
# .plot(1) <- this plots the y component

Show your work:

## 4. Simulating the Elastic Pendulum
Now that we know our simulation is working, let's go ahead and model an elastic pendulum. Choose reasonable parameters and initial conditions based on your new experience with the pendulum system. If you're unsure, consult the previous exercise for inspiration.

In [ ]:
# parameters and initial conditions
m = 
k = 
g = 
rb =
L = 
dt = 
N = 
r0 = 
v0 = 

# simulate motion

# plot motion

### Animating the motion
Animate an elastic pendulum using the sample code in `single_particle_dynamics.py`. Feel free to mess around with the parameters and the initial conditions. Create 3 different animations and save them under a folder called `animations`.

**Note**: Animations are literally just a sequence of still images. Considering the time it takes for creating a singe image, creating 10,000 will obviously take some time. If it takes too long to save the animations, you can just show them here.

In [ ]:
# animation 1

In [ ]:
# animation 2

In [ ]:
# animation 3